In [73]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

In [74]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000453273s, CPU 0.000454s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00902188s, CPU 0.008769s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [75]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:AxiSymmetricHeatConduction)

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 16)


LowLevelFEM.Problem("body1-2D", :AxiSymmetricHeatConduction, 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666)], 1.0, 98)

In [76]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=-1)

("right", 1, 0, 0)

In [77]:
T0 = FEM.initialTemperature(problem, "body", T=0)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T)

In [78]:
FEM.initialTemperature!(problem, "left", T0, T=0)
T0

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.0; … ; 0.0; 0.0;;], [0.0], Int64[], 1, :T)

In [79]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0; 0.39269908169872364; … ; 0.0; 0.0;;], Float64[], Int64[], 1, :qn)

In [80]:
FEM.applyBoundaryConditions!(problem, K, C, q, [supp])

In [81]:
λmax = FEM.largestEigenValue(K, C)

28621.755935725072

In [82]:
ϑ = 0.33
T = FEM.FDM(K, C, q, T0, 200 * (2 / ((1 - 2ϑ) * λmax)), 10 * 2 / ((1 - 2ϑ) * λmax), ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.006401883630411036 … -5.85711280492516 10.571854980157749; … ; 0.0 3.635616781098292e-6 … -0.03858304243429837 0.12291706161444978; 0.0 0.00046848737297135434 … 0.25827905139774654 -1.0360896183999244], [0.0, 0.002055203375497393, 0.004110406750994786, 0.006165610126492179, 0.008220813501989572, 0.010276016877486964, 0.012331220252984356, 0.014386423628481748, 0.01644162700397914, 0.018496830379476532, 0.020552033754973924, 0.022607237130471317, 0.02466244050596871, 0.0267176438814661, 0.028772847256963493, 0.030828050632460885, 0.03288325400795828, 0.034938457383455676, 0.03699366075895307, 0.03904886413445047], Int64[], 20, :T)

In [83]:
ϑ = 0.66
T = FEM.FDM(K, C, q, T0, 0.15, 0.01, ϑ=ϑ)

LowLevelFEM.ScalarField(Matrix{Float64}[], [0.0 0.0 … 0.0 0.0; 0.0 0.012049116727636457 … 0.06119482182241499 0.06364298154325056; … ; 0.0 0.0019286343562229657 … 0.03925976440258409 0.04111402895428547; 0.0 0.006117427403266316 … 0.05540770953607672 0.057807165266333026], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :T)

In [84]:
FEM.showDoFResults(problem, T, :T, visible=true)

0

In [85]:
q1 = FEM.solveHeatFlux(problem, T)

LowLevelFEM.VectorField([[0.0 -0.926251525698612 … -1.0449453710312078 -1.056272365565014; 0.0 0.004348674571823807 … 1.7232144908507774e-5 -6.211178650905591e-5; … ; 0.0 -0.926251525698612 … -1.0449453710312078 -1.056272365565014; 0.0 0.004348674571823807 … 1.7232144908507774e-5 -6.211178650905591e-5], [0.0 -0.9276687164204831 … -1.045513656994826 -1.0568674646694336; 0.0 0.006507219132264597 … 0.0008827970661334783 0.0008442926338259049; … ; 0.0 -0.9276687164204831 … -1.045513656994826 -1.0568674646694336; 0.0 0.006507219132264597 … 0.0008827970661334783 0.0008442926338259049], [0.0 -0.9055963282633359 … -1.0492698368214473 -1.061238047633619; 0.0 -0.021325348663103594 … 0.0037099083482442646 0.00428358765005972; … ; 0.0 -0.9055963282633359 … -1.0492698368214473 -1.061238047633619; 0.0 -0.021325348663103594 … 0.0037099083482442646 0.00428358765005972], [0.0 -0.20129504926463487 … -3.2610903824095296 -3.4114125642085478; 0.0 0.003691560633160302 … 0.27179270812031947 0.285491898023148

In [86]:
FEM.probe(q1, 0.5, 0.5, 0, step=10)

2-element Vector{Float64}:
 -1.1623763042734412
  0.024203407331050975

In [87]:
FEM.showHeatFluxResults(problem, q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


1

In [88]:
q2 = FEM.solveHeatFlux(problem, T, DoFResults=true)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -0.5091861085381506 … -11.465195779078526 -12.011638893476269; 0.0 -0.03827341865941146 … -0.9133922235617322 -0.9571420731938983; … ; 0.0 -0.7252650894302866 … -1.0960809113103618 -1.1146254571866778; 0.0 -0.0013271185787032136 … 0.0014013941723005352 0.0014951150627884324], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D)

In [89]:
FEM.showDoFResults(problem, q2, :qvec)

2

In [90]:
FEM.probe(q2, 0.5, 0.5, 0, step=10)

2-element Vector{Float64}:
 -1.1779798215478603
  0.0033510974104878377

In [91]:
q3 = FEM.elementsToNodes(problem, q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 -0.5091861085381506 … -11.465195779078526 -12.011638893476269; 0.0 -0.03827341865941146 … -0.9133922235617322 -0.9571420731938983; … ; 0.0 -0.7252650894302866 … -1.0960809113103618 -1.1146254571866778; 0.0 -0.0013271185787032136 … 0.0014013941723005352 0.0014951150627884324], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D)

In [92]:
FEM.showDoFResults(problem, q3, :qvec)

3

In [93]:
err1 = FEM.fieldError(problem, q1)

LowLevelFEM.VectorField(Matrix{Float64}[], [0.0 0.03827341865941153 … 0.9133922235617327 0.9571420731938982; 0.0 0.03827341865941146 … 0.9133922235617322 0.9571420731938983; … ; 0.0 0.11069112254051344 … 0.03327325845092234 0.03769094215071261; 0.0 0.04753309912145688 … 0.016031839293501358 0.018026162125151766], [0.0, 0.01, 0.02, 0.03, 0.04, 0.05, 0.060000000000000005, 0.07, 0.08, 0.09, 0.09999999999999999, 0.10999999999999999, 0.11999999999999998, 0.12999999999999998, 0.13999999999999999], Int64[], 15, :q2D)

In [94]:
FEM.showDoFResults(problem, err1, :vector)

4

In [95]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [96]:
gmsh.finalize()